<a href="https://colab.research.google.com/github/jan-kreischer/UZH_ML4NLP/blob/main/Project-05/ex05_sent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 5 - Sequence and Sentiment Classification using Transformers

## Part 2: Resource Limited Competition: Sentiment Analysis

## 1. Setup
### 1.1 Dependencies
Disclaimer: The output of cells which do not produce not helpful output (for example the pip install comands) were cleared to make the program easier to read

In [1]:
!pip install datasets transformers sklearn simpletransformers

     |████████████████████████████████| 298 kB 4.3 MB/s 
     |████████████████████████████████| 3.1 MB 83.0 MB/s 
     |████████████████████████████████| 247 kB 74.9 MB/s 
     |████████████████████████████████| 1.1 MB 63.4 MB/s 
     |████████████████████████████████| 132 kB 88.8 MB/s 
     |████████████████████████████████| 243 kB 89.8 MB/s 
     |████████████████████████████████| 59 kB 8.1 MB/s 
     |████████████████████████████████| 596 kB 65.1 MB/s 
     |████████████████████████████████| 3.3 MB 53.7 MB/s 
     |████████████████████████████████| 895 kB 57.5 MB/s 
     |████████████████████████████████| 9.1 MB 58.4 MB/s 
     |████████████████████████████████| 1.2 MB 64.4 MB/s 
     |████████████████████████████████| 1.7 MB 56.8 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 97 kB 8.2 MB/s 
     |████████████████████████████████| 180 kB 57.4 MB/s 
     |████████████████████████████████| 140 kB 69.7 MB/s 
     |███████████████


### 1.2 Imports

In [2]:
import datasets
from datasets import load_dataset
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder

# Misc
import os
import shutil
import csv
import re
from io import StringIO
import requests
import string
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sn

# Pandas
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Keras
import keras.preprocessing
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, AveragePooling1D, Dense, Dropout, Activation, Embedding
from keras import backend as K
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# tensorflow
import tensorflow as tf

# Torch
import torch

# Sklearn
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

### 1.3 Constants

In [3]:
train_range=(10000,15000)
test_range=(11500,13500)

### 1.4 Environment
We check if the environment we are using is properly setup, such that we are using GPU for training our models.

In [4]:
# Check if device supports CUDA interface
CUDA = torch.cuda.is_available()
# Make program run on gpu (cuda:0) if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu:0')
torch.cuda.set_device(device)
print('Using device:', device)

Using device: cuda:0


In [5]:
# Check and print information about available GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 29 19:56:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# Get GPU name
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-1fbe673f-4fb4-9506-1c0a-a08fad919fd9)


In [7]:
# Check Memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## Import


Here we import the data from the Stanford Repository.

In [8]:
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

84140032/84125825 [==============================] - 5s 0us/step


After download the data from the above link, we found that the directory sturcture is like this:

main_directory/                 
...train/                
......a_text_1.txt                
......a_text_2.txt                
...test/                
......a_text_1.txt                
......a_text_2.txt                
...unsup/                
......                

We formalize the path to the main directory and its subdirectory. We also remove the "unsup" directory which contains unlabeled reviews for unsupervised learning.

In [9]:
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
train_dir = os.path.join(main_dir, 'train')
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)   

In [10]:
# read data into dataframe: train_data. According to the requeirement, we read train[10000:15000] as train_data.
train = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=25000,
    shuffle=False,
    validation_split=0)

for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
train_data=train[train_range[0]:train_range[1]]

Found 25000 files belonging to 2 classes.


In [11]:
# similarly read the test data into dataframe. According to the requeirement, we read test[11500:13500] as test_data.
test = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=25000,
    shuffle=False,
    validation_split=0)

for i in test.take(1):
  test_feat = i[0].numpy()
  test_lab = i[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
test_data=test[test_range[0]:test_range[1]]

Found 25000 files belonging to 2 classes.


In [12]:
print("Train_data has a shape of {}. \n\n The number of positive(1) and negative(0) reviews are:\n {}".format(
    train_data.shape,train_data['LABEL_COLUMN'].value_counts()))

Train_data has a shape of (5000, 2). 

 The number of positive(1) and negative(0) reviews are:
 1    2500
0    2500
Name: LABEL_COLUMN, dtype: int64


In [13]:
print("Test_data has a shape of {}. \n\n The number of positive(1) and negative(0) reviews are:\n {}".format(
    test_data.shape,test_data['LABEL_COLUMN'].value_counts()))

Test_data has a shape of (2000, 2). 

 The number of positive(1) and negative(0) reviews are:
 1    1000
0    1000
Name: LABEL_COLUMN, dtype: int64


In [14]:
train_data

,DATA_COLUMN,LABEL_COLUMN
10000,"First, the CGI in this movie was horrible. I w...",0
10001,The film is about a sabretooth on the lose at ...,0
10002,Everything about this film is hog wash. Pitifu...,0
10003,Spoilers will be in this. The movie could have...,0
10004,Three giant sabretooth tigers(..created in a l...,0
...,...,...
14995,The minute I started watching this I realised ...,1
14996,i really loved this version of Emma the best. ...,1
14997,Until the 1990s there had never been a film ba...,1
14998,Old Jane's mannered tale seems very popular th...,1


In [15]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [16]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args1 = ClassificationArgs(num_train_epochs=5,
                                use_early_stopping=True,
                                output_dir="outputs1/",
                                overwrite_output_dir=True)

# Create a ClassificationModel
model1 = ClassificationModel(
    "distilbert", 
    "distilbert-base-uncased-finetuned-sst-2-english", 
    args=model_args1,
    #num_labels=2,
    #weight=[1,1]
)

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [17]:
model1.train_model(train_data)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_distilbert_128_2_2


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/625 [00:00<?, ?it/s]

...

Running Epoch 2 of 5:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/625 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs1/.


(3125, 0.1718678014612198)

In [19]:
# Evaluate the model
result, model_outputs, wrong_predictions = model1.eval_model(test_data)
print(result, model_outputs, wrong_predictions)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_distilbert_128_2_2


Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.6992549248886533, 'tp': 863, 'tn': 836, 'fp': 164, 'fn': 137, 'auroc': 0.9174225, 'auprc': 0.9031636814896324, 'eval_loss': 1.1014629443287849}


{'mcc': 0.6992549248886533, 'tp': 863, 'tn': 836, 'fp': 164, 'fn': 137, 'auroc': 0.9174225, 'auprc': 0.9031636814896324, 'eval_loss': 1.1014629443287849} [[ 4.03125    -3.69921875]
 [ 4.65234375 -4.16796875]
 [ 3.16992188 -2.90820312]
 ...
 [-4.72265625  5.1171875 ]
 [-4.5078125   4.7890625 ]
 [-4.703125    4.9765625 ]] []


In [20]:
# Make predictions with the model
predictions, raw_outputs = model1.predict(["The movie is sucks"])
print(predictions, raw_outputs)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[0] [[ 4.57421875 -4.0703125 ]]


In [21]:
# Make predictions with the model
predictions, raw_outputs = model1.predict(["The movie is great"])
print(predictions, raw_outputs)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1] [[-4.73828125  5.1875    ]]


In [22]:
# Make predictions with the model
predictions, raw_outputs = model1.predict(["The movie is so so"])
print(predictions, raw_outputs)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[0] [[ 2.48828125 -2.27539062]]


Here we import some other pretrained models, and tokenizers and later we will compare them to find out the best one.

# Not relevant anymore

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model1 = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=3)
tokenizer1 = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased',num_labels=3)

## Tokenization
Here we use the tokenizer imported above to encode the corpus.

In [ ]:
reviews=train_data['DATA_COLUMN'].values.tolist()
labels=train_data['LABEL_COLUMN'].values.tolist()
reviews_test=test_data['DATA_COLUMN'].values.tolist()

train_encodings = tokenizer1(reviews, truncation=True, padding=True,return_tensors = 'pt')
test_encodings = tokenizer1(reivews_test, truncation=True, padding=True,return_tensors = 'pt')

Define torch datasets objects.

In [ ]:
# Train Dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


# Test Dataset
class SentimentTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return len(self.encodings)

## Data Loaders

In [ ]:
train_dataset = SentimentDataset(train_encodings, labels)
test_dataset = SentimentTestDataset(test_encodings)

## Fine-tuning with Trainer

Define a Metrics Function

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #recall = recall_score(y_true=labels, y_pred=pred)
    #precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(labels, pred, average='weighted')

    return {"accuracy": accuracy,"f1_score":f1}

Define Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir='./res',          # output directory
    evaluation_strategy="steps",
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    #logging_dir='./logs4',            # directory for storing logs
    #logging_steps=10,
    #load_best_model_at_end=True,
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model1,# model imported above
    args=training_args, # training arguments, defined above
    train_dataset=train_dataset,# training dataset
    compute_metrics=compute_metrics,
)

trainer.train()

***** Running training *****
  Num examples = 5000
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 785


RuntimeError: ignored